In [ ]:
import pandas as pd
data = pd.read_parquet('/drive2/phatnt/zTrans/data/data_method_tresitter1.parquet',index = False)

In [2]:
print(data.iloc[1])

fromLib                                                           ant:ant
toLib                                                  org.apache.ant:ant
repoName                                              bobmcwhirter_drools
prevCommit                       2ba5d35fb486c4a16b5b8b15fc247e8759bfa54f
startCommit                      92384035a8651b675c82689b24837eb8adb81d66
endCommit                        0df25aead759b3b542a685ac21a3a009bcd22fe9
fileName                drools-compiler/src/test/java/org/drools/guvno...
startCode               package org.drools.guvnor.server.util;\n\nimpo...
endCode                 package org.drools.guvnor.server.util;\n\nimpo...
startCommitChanges                               +org.mockito:mockito-all
endCommitChanges                -org.jmock:jmock\n-org.jmock:jmock-legacy
startCode_cleaned       package org.drools.guvnor.server.util;\nimport...
endCode_cleaned         package org.drools.guvnor.server.util;\nimport...
diff_cleaned            --- \n+++ \n@@

In [3]:
data_before_null = data[data['methods_before'] == '']
data_after_null = data[data['methods_after'] == '']
special = pd.concat([data_before_null, data_after_null], ignore_index=True)
print('Co :',len(special) *100 / len(data) , '% la special')

Co : 52.99747311368547 % la special


In [3]:
import difflib
def get_diff_lines(method_1,method_2):
    diff = difflib.unified_diff(
    method_1.splitlines(), 
    method_2.splitlines(), 
    lineterm='',  # Avoid newline being added to the output
    fromfile='Method 1',
    tofile='Method 2'
    )
    diff = '\n'.join(diff)
    diff_changes = diff
    diff = diff.splitlines()
    
    remove = 0
    add = 0
    for i in diff:
        if i.startswith('-') and '---' not in i:
            remove+=1
        elif i.startswith('+') and '+++' not in i:
            add += 1
    return {'remove' : remove,'add' : add,'diff_changes' : diff_changes}

In [4]:
data[['remove_line', 'add_line','diff_changes']] = data.apply(
    lambda row: pd.Series(get_diff_lines(row['methods_before'], row['methods_after'])),
    axis=1
)


In [ ]:
data.to_parquet('/drive2/phatnt/zTrans/data/data_method_tresitter1.parquet')

: 

In [ ]:
# loại bỏ các special
rows_to_remove = pd.concat([data_before_null, data_after_null])

data = data[~data.index.isin(rows_to_remove.index)]

In [ ]:
special = data[(data['methods_before'] == '') | (data['methods_after'] == '')]
total_counts = data.groupby(["startCommit", "endCommit"]).size().reset_index(name="total_count")
special_counts = special.groupby(["startCommit", "endCommit"]).size().reset_index(name="special_count")
summary = pd.merge(total_counts, special_counts, on=["startCommit", "endCommit"], how="left").fillna(0)
summary["special_ratio"] = summary["special_count"] / summary["total_count"]
summary

,startCommit,endCommit,total_count,special_count,special_ratio
0,00e9a259ab6d8bde0456bdf168fed66a7cedbcff,00e9a259ab6d8bde0456bdf168fed66a7cedbcff,2,0.0,0.000000
1,0188e9aa280b800710848d68a93af4cb28b050da,0188e9aa280b800710848d68a93af4cb28b050da,289,92.0,0.318339
2,019d1e6e6e3715b2be91b01a48a3bd4d14a5b216,019d1e6e6e3715b2be91b01a48a3bd4d14a5b216,2,0.0,0.000000
3,01ae58600c4069cd4df5081d52e81e05411694d9,01ae58600c4069cd4df5081d52e81e05411694d9,3,0.0,0.000000
4,01ca345463fdaee4d5d72293bb7679fb5bc815ec,01ca345463fdaee4d5d72293bb7679fb5bc815ec,1,1.0,1.000000
...,...,...,...,...,...
869,fe5c7d3f17529442f7a62355a92d51a294ad2769,fe5c7d3f17529442f7a62355a92d51a294ad2769,99,28.0,0.282828
870,fee22c96dcbf09bf8d0ea5dff58a9191e04b4398,fee22c96dcbf09bf8d0ea5dff58a9191e04b4398,8,4.0,0.500000
871,ff048221f4cbcbc885343461a2572404c6020753,ff048221f4cbcbc885343461a2572404c6020753,282,142.0,0.503546
872,ff661a403578c638852652ee93a5e47f447903f9,ff661a403578c638852652ee93a5e47f447903f9,3,0.0,0.000000


In [12]:
# Tbinh có khoảng 50 methods special
summary['special_count'].describe()

count      874.000000
mean        49.673913
std        421.569098
min          0.000000
25%          0.000000
50%          3.000000
75%         13.000000
max      10373.000000
Name: special_count, dtype: float64

In [ ]:
# Tbinh trong 1 migration thì có 31% là special
summary['special_ratio'].describe()

count    874.000000
mean       0.318187
std        0.321754
min        0.000000
25%        0.000000
50%        0.241212
75%        0.565600
max        1.000000
Name: special_ratio, dtype: float64

In [ ]:
# 1 class có tầm 75% là thay đổi dưới 5 methods, trung bình thay đổi 5 methods
method_counts = data.groupby("startCode").size().reset_index(name="methods_before_count")
method_counts['methods_before_count'].describe()

count    15490.000000
mean         5.288509
std         11.751422
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max        728.000000
Name: methods_before_count, dtype: float64

In [ ]:
# tbinh 1 method thay đôi khoảng 10 dòng
data['changes_line'] = data.apply(lambda x: x['remove_line'] + x['add_line'], axis=1)
data['changes_line'].describe()

count    81919.000000
mean        10.324369
std         15.041808
min          0.000000
25%          3.000000
50%          6.000000
75%         12.000000
max        522.000000
Name: changes_line, dtype: float64

In [13]:
t = data[data['changes_line'] == 522]
for i in t['methods_before']:
    print(i)

@SuppressWarnings({"fallthrough", "nls"})
  public boolean doPhase1(ASTNode ast, QB qb, Phase1Ctx ctx_1, PlannerContext plannerCtx)
      throws SemanticException {
    boolean phase1Result = true;
    QBParseInfo qbp = qb.getParseInfo();
    boolean skipRecursion = false;
    if (ast.getToken() != null) {
      skipRecursion = true;
      switch (ast.getToken().getType()) {
      case HiveParser.TOK_SELECTDI:
        qb.countSelDi();
      case HiveParser.TOK_SELECT:
        qb.countSel();
        qbp.setSelExprForClause(ctx_1.dest, ast);
        int posn = 0;
        if (((ASTNode) ast.getChild(0)).getToken().getType() == HiveParser.TOK_HINTLIST) {
          qbp.setHints((ASTNode) ast.getChild(0));
          posn++;
        }
        if ((ast.getChild(posn).getChild(0).getType() == HiveParser.TOK_TRANSFORM))
          queryProperties.setUsesScript(true);
        LinkedHashMap<String, ASTNode> aggregations = doPhase1GetAggregationsFromSelect(ast,
            qb, ctx_1.dest);
        d